## TUTORIAL 14 - Stokes Optimal Control
*__Keywords: distributed optimal control, geometrical parametrization, inf-sup condition, POD-Galerkin__*

### 1. Introduction

In this tutorial, we consider a distributed optimal control problem for a Couette flow using the Stokes equations in a two-dimensional domain $\Omega_o(\boldsymbol{\mu})$ shown below:

<img src="data/mesh1.png" width="50%"/>

The problem is characterized by two parameters, $\mu_0$ and $\mu_1$. The first parameter, $\mu_0$, is a geometrical parameter that describes the channel length where $\mu_0 \in [0.5,2]$. The second parameter, $\mu_1$, is a physical parameter in the forcing term of the state equation where $\mu_1 \in [0.5,1.5]$. 

Thus, the parameter vector $\boldsymbol{\mu}$ is given by: $$\boldsymbol{\mu}=(\mu_0,\mu_1)$$ on the parameter domain $$\mathbb{P}=[0.5,2] \times [0.5,1.5].$$

In order to obtain a faster approximation of the optimal control problem, without any remeshing, we pursue an optimize-then-discretize approach using the POD-Galerkin method from a fixed, parameter-independent reference domain $\Omega$.

### 2. Parametrized formulation

Let $\boldsymbol{v_o}(\boldsymbol{\mu})$ represent the velocity field in the channel, $p_o(\boldsymbol{\mu})$ represent the pressure in the domain $\Omega_o(\boldsymbol{\mu})$. Let $\boldsymbol{u}(\boldsymbol{\mu})$ characterize the control vector-valued function.

Consider the following optimal control problem:
$$
\underset{\boldsymbol{u}_o \in U_o}{min} \; J(\boldsymbol{v_o}(\boldsymbol{\mu}), p_o(\boldsymbol{\mu}), \boldsymbol{u_o}(\boldsymbol{\mu})) = \frac{1}{2} \left\lVert v_{o1}(\boldsymbol{\mu}) - x_{o2} \right\rVert^2_{L^2(\Omega_o)} + \frac{\alpha}{2} \left\lVert \boldsymbol{u_o}(\boldsymbol{\mu}) \right\rVert^2_{L^2(\Omega_o)}
$$

$$ 
\text{s.t.} 
\begin{cases}
    -\nu \Delta \boldsymbol{v}_o + \nabla p_o =  \boldsymbol{f}_o(\boldsymbol{\mu}) + \boldsymbol{u}_o \qquad \quad \text{in} \; \Omega_o(\boldsymbol{\mu}) \\
    \text{div} \, \boldsymbol{v}_o = 0 \qquad \qquad \qquad \qquad \quad \; \text{in} \; \Omega_o(\boldsymbol{\mu}) \\
    v_{o1} = x_{o2}, v_{o2} = 0 \qquad \qquad \qquad \; \, \text{on} \; \Gamma_D^o(\boldsymbol{\mu}) \\
    -p_o \boldsymbol{n}_{o1} + \nu \frac{\partial v_{o1}}{\partial \boldsymbol{n}_{o1}} = 0, v_{o2}=0 \qquad \; \, \, \text{on} \;\Gamma_N^o(\boldsymbol{\mu})
\end{cases}
$$

where
* $\nu$ represents kinematic viscosity
* the forcing term is given by $\boldsymbol{f}_o(\boldsymbol{\mu}) = (0, -\mu_1)$
* we observe only the first component of the velocity observation function, which is equal to $x_{o2}$
* the velocity space is defined as $\mathbb{V}_o = [H^1_{\Gamma_D}(\Omega_o)]^2$
* the pressure space is defined as $\mathbb{M}_o = L^2(\Omega_o)$ 
* the state space is defined as $\mathbb{Y}_o = \mathbb{V}_o \times \mathbb{M}_o$ 
* the adjoint space is defined as $\mathbb{Q}_o = \mathbb{Y}_o$ 
* the control space is defined as $\mathbb{U}_o = [L^2(\Omega_o)]^2$

The corresponding weak formulation of the optimal control problem is derived from solving the respective Lagrangian functionals for each equation in the system. Let $\boldsymbol{w}$ and $q$ represent the Lagrange multiplier for the functional involving the equation of motion of flow and incompressibility constraint, respectively. 

Solving the problem

$$
\text{find} \; (\boldsymbol{v}_o, p_o, \boldsymbol{w}_o, \boldsymbol{u}_o) \in \mathbb{Y} \times \mathbb{Q} \times \mathbb{U} \, : \\ \nabla L_{o1}(\boldsymbol{v}_o, p_o, \boldsymbol{w}_o, \boldsymbol{u}_o)[\boldsymbol{\psi},\pi,\boldsymbol{\phi},\boldsymbol{\tau}] = 0 \quad \forall (\boldsymbol{\psi},\pi,\boldsymbol{\phi},\boldsymbol{\tau}) \in \mathbb{Y} \times \mathbb{Q} \times \mathbb{U} 
$$

and 

$$
\text{find} \; (\boldsymbol{v}_o, q_o, \boldsymbol{u}_o) \in \mathbb{Y} \times \mathbb{U} \, : \\ \nabla L_{o2}(\boldsymbol{v}_o, q_o, \boldsymbol{u}_o)[\boldsymbol{\psi},\xi,\boldsymbol{\tau}] = 0 \quad \forall (\boldsymbol{\psi},\xi,\boldsymbol{\tau}) \in \mathbb{Y} \times \mathbb{U} 
$$

gives the weak formulation:
$$
\begin{cases}
    L_{o1,\boldsymbol{v}} = m_o(\boldsymbol{v}_o,\boldsymbol{\psi}_o) - g_o(\boldsymbol{v_d}, \boldsymbol{\psi}_o) - a^*_o(\boldsymbol{w}_o,\boldsymbol{\psi}_o) \\
    L_{o1,p} = b^*_o(\pi_o, \boldsymbol{w}_o) \\
    L_{o1,\boldsymbol{w}} = -a_o(\boldsymbol{v}_o,\boldsymbol{\phi}_o)+ b^T_o(p_o, \boldsymbol{phi}_o) + f_o(\boldsymbol{\phi}_o) + c_o(\boldsymbol{u}_o,\boldsymbol{\phi}_o) \\
    L_{o1, \boldsymbol{u}} = \alpha n_o(\boldsymbol{u}_o,\boldsymbol{\tau}_o) + c^*_o(\boldsymbol{\tau}_o,\boldsymbol{w}_o) \\
    L_{o2, \boldsymbol{v}} = m_o(\boldsymbol{v}_o,\boldsymbol{\psi}_o) - g_o(\boldsymbol{v_d}, \boldsymbol{\psi}_o) + b^{*T}_o(q_o, \boldsymbol{\psi}_o) \\
    L_{o2, q} = b_o(\xi_o, \boldsymbol{v}_o) \\
\end{cases}
$$



In [ ]:
from dolfin import *
from rbnics import *

### 3. Affine Decomposition

In order to obtain an affine decomposition, we recast the problem on a fixed, parameter independent, reference domain Ω. We choose the reference domain characterized by $\boldsymbol{\mu}_{ref}=(1,1)$ which we generate through the generate_mesh notebook provided in the data folder.

In [ ]:
@PullBackFormsToReferenceDomain()
@ShapeParametrization(
    ("x[0]", "mu[0] * x[1]"),  # subdomain 1
)
class StokesOptimalControl(StokesOptimalControlProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        StokesOptimalControlProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        trial = TrialFunction(V)
        (self.v, self.p, self.u, self.w, self.q) = split(trial)
        test = TestFunction(V)
        (self.psi, self.pi, self.tau, self.phi, self.xi) = split(test)
        self.dx = Measure("dx")(subdomain_data=subdomains)
        self.ds = Measure("ds")(subdomain_data=boundaries)
        # Regularization coefficient
        self.alpha = 0.008
        # Constant viscosity
        self.nu = 0.1
        # Desired velocity
        self.vx_d = Expression("x[1]", degree=1)
        # Customize linear solver parameters
        self._linear_solver_parameters.update({
            "linear_solver": "mumps"
        })

    # Return custom problem name
    def name(self):
        return "StokesOptimalControl1"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_supremizers
    def compute_theta(self, term):
        mu = self.mu
        if term in ("a", "a*"):
            theta_a0 = self.nu * 1.0
            return (theta_a0,)
        elif term in ("b", "b*", "bt", "bt*"):
            theta_b0 = 1.0
            return (theta_b0,)
        elif term in ("c", "c*"):
            theta_c0 = 1.0
            return (theta_c0,)
        elif term == "m":
            theta_m0 = 1.0
            return (theta_m0,)
        elif term == "n":
            theta_n0 = self.alpha * 1.0
            return (theta_n0,)
        elif term == "f":
            theta_f0 = - mu[1]
            return (theta_f0,)
        elif term == "g":
            theta_g0 = 1.0
            return (theta_g0,)
        elif term == "l":
            theta_l0 = 1.0
            return (theta_l0,)
        elif term == "h":
            theta_h0 = 1.0
            return (theta_h0,)
        elif term == "dirichlet_bc_v":
            theta_bc0 = mu[0]
            return (theta_bc0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_supremizers
    def assemble_operator(self, term):
        dx = self.dx
        if term == "a":
            v = self.v
            phi = self.phi
            a0 = inner(grad(v), grad(phi)) * dx
            return (a0,)
        elif term == "a*":
            psi = self.psi
            w = self.w
            as0 = inner(grad(w), grad(psi)) * dx
            return (as0,)
        elif term == "b":
            xi = self.xi
            v = self.v
            b0 = - xi * div(v) * dx
            return (b0,)
        elif term == "bt":
            p = self.p
            phi = self.phi
            bt0 = - p * div(phi) * dx
            return (bt0,)
        elif term == "b*":
            pi = self.pi
            w = self.w
            bs0 = - pi * div(w) * dx
            return (bs0,)
        elif term == "bt*":
            q = self.q
            psi = self.psi
            bts0 = - q * div(psi) * dx
            return (bts0,)
        elif term == "c":
            u = self.u
            phi = self.phi
            c0 = inner(u, phi) * dx
            return (c0,)
        elif term == "c*":
            tau = self.tau
            w = self.w
            cs0 = inner(tau, w) * dx
            return (cs0,)
        elif term == "m":
            v = self.v
            psi = self.psi
            m0 = v[0] * psi[0] * dx
            return (m0,)
        elif term == "n":
            u = self.u
            tau = self.tau
            n0 = inner(u, tau) * dx
            return (n0,)
        elif term == "f":
            phi = self.phi
            f0 = phi[1] * dx
            return (f0,)
        elif term == "g":
            psi = self.psi
            vx_d = self.vx_d
            g0 = vx_d * psi[0] * dx
            return (g0,)
        elif term == "l":
            xi = self.xi
            l0 = Constant(0.0) * xi * dx
            return (l0,)
        elif term == "h":
            vx_d = self.vx_d
            h0 = vx_d * vx_d * dx(domain=mesh)
            return (h0,)
        elif term == "dirichlet_bc_v":
            bc0 = [DirichletBC(self.V.sub("v").sub(0), self.vx_d, self.boundaries, 1),
                   DirichletBC(self.V.sub("v").sub(1), Constant(0.0), self.boundaries, 1)]
            return (bc0,)
        elif term == "dirichlet_bc_w":
            bc0 = [DirichletBC(self.V.sub("w"), Constant((0.0, 0.0)), self.boundaries, 1)]
            return (bc0,)
        elif term == "inner_product_v":
            v = self.v
            psi = self.psi
            x0 = inner(grad(v), grad(psi)) * dx
            return (x0,)
        elif term == "inner_product_p":
            p = self.p
            pi = self.pi
            x0 = p * pi * dx
            return (x0,)
        elif term == "inner_product_u":
            u = self.u
            tau = self.tau
            x0 = inner(u, tau) * dx
            return (x0,)
        elif term == "inner_product_w":
            w = self.w
            phi = self.phi
            x0 = inner(grad(w), grad(phi)) * dx
            return (x0,)
        elif term == "inner_product_q":
            q = self.q
            xi = self.xi
            x0 = q * xi * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main Program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [ ]:
mesh = Mesh("data/mesh1.xml")
subdomains = MeshFunction("size_t", mesh, "data/mesh1_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/mesh1_facet_region.xml")

### 4.2. Create Finite Element space (P2-P1 Taylor-Hood)

In [ ]:
velocity_element = VectorElement("Lagrange", mesh.ufl_cell(), 2)
pressure_element = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
element = MixedElement(velocity_element, pressure_element, velocity_element, velocity_element, pressure_element)
V = FunctionSpace(mesh, element, components=[["v", "s"], "p", "u", ["w", "r"], "q"])

### 4.3. Allocate an object of the StokesOptimalControl class

In [ ]:
problem = StokesOptimalControl(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.5, 2.0), (0.5, 1.5)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with POD-Galerkin method

In [ ]:
pod_galerkin_method = PODGalerkin(problem)
pod_galerkin_method.set_Nmax(10)

### 4.5. Perform the offline phase

In [ ]:
lifting_mu = (1.0, 1.0)
problem.set_mu(lifting_mu)
pod_galerkin_method.initialize_training_set(100)
reduced_problem = pod_galerkin_method.offline()

### 4.6. Perform an online solve

In [ ]:
online_mu = (1.7, 1.5)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
print("Reduced output for mu =", online_mu, "is", reduced_problem.compute_output())

In [ ]:
plot(reduced_solution, reduced_problem=reduced_problem, component="v")

In [ ]:
plot(reduced_solution, reduced_problem=reduced_problem, component="p")

In [ ]:
plot(reduced_solution, reduced_problem=reduced_problem, component="u")

In [ ]:
plot(reduced_solution, reduced_problem=reduced_problem, component="w")

In [ ]:
plot(reduced_solution, reduced_problem=reduced_problem, component="q")

### 4.7. Perform an error analysis

In [ ]:
pod_galerkin_method.initialize_testing_set(100)
pod_galerkin_method.error_analysis()

### 4.8. Perform a speedup analysis

In [ ]:
pod_galerkin_method.speedup_analysis()